In [2]:
from __future__ import print_function, division
!pip install tensorflow

import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import torch
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

import torch.backends.cudnn as cudnn
cudnn.benchmark = True
print("GPU: " + str(torch.cuda.is_available()))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
GPU: True


In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Load the CIFAR100 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert pixel values to float data type and normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

data = []
data.extend(x_train)
data.extend(x_test)
data_min = np.min(data, axis=(1,2), keepdims=True)
data_max = np.max(data, axis=(1,2), keepdims=True)

scaled_data = (data - data_min) / (data_max - data_min)

#push normalized data back to x_train and x_test
x_train = scaled_data[:len(x_train)]
x_test = scaled_data[len(x_train):]

# make labels categorical
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


170498071/170498071 [==============================] - 2s 0us/step


In [4]:
#from chat gpt
from tensorflow.keras.applications.resnet50 import ResNet50
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = resnet_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=resnet_model.input, outputs=predictions)


94765736/94765736 [==============================] - 2s 0us/step


In [ ]:
batch_size = 100
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#go to github and use a simple resnet/alexnet as a benchmark  

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])
model.summary()

Epoch 1/100
500/500 [==============================] - 80s 66ms/step - loss: 1.0932 - accuracy: 0.6407 - val_loss: 3.1230 - val_accuracy: 0.1612
Epoch 2/100
105/500 [=====>........................] - ETA: 23s - loss: 0.7671 - accuracy: 0.7408

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
import matplotlib.pyplot as plt

plt.plot(train_acc)
plt.plot(val_acc)
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(train_loss)
plt.plot(val_loss)
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
pred_train= model.predict(x_train)
scores = model.evaluate(x_train, y_train, verbose=0)
print('Accuracy on training data: {} \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(x_test)
scores2 = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy on test data: {} \n Error on test data: {}'.format(scores2[1], 1 - scores2[1])) 

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)